In [1]:
from carbon import CarbonSimulatorUI, __version__, __date__
from collections import namedtuple
print("[carbon] version", __version__, __date__)

[carbon] version 1.1 19/Nov/2022


# Prices and Slippage

## Introduction and definitions

This workbook serves to define certain quantities, and explains how to exactly compute them. As context, we are operating within an AMM framework where $x,y$ are the respective token balances. Depending on the price convention of the pair, prices are either expressed in $x/y$ or in $y/x$. We here assume the latter. For the opposite convention, $x,y$ need to be reversed in _all_ formulas below.

- **Effective Price**. The _effective price_ is the overall price obtained by a specific trade of a specific size. Specifically, $P_{eff}(\Delta y) = - \Delta x / \Delta y$. Note that prices are always positive but we assume that outflows (from the AMM point of view) have a negative sign, inflows a positive one.

- **Marginal Price** or **Current Price**. The _marginal price_ aka _current price_ is the effective price for a very small trade; formally we define that the marginal / current price is given as $P_{marg}^\pm = P_{curr}^\pm = P_{eff}(\Delta x^\pm \rightarrow 0^\pm)$; the $\pm$ sign (that we will omit below) reminds us that the marginal price will be different on the bid and on the ask side.

- **Price Impact**. We define* the _price impact_ as the difference (or percentace change) of the marginal prices before or after that trade; formally we define it as $P_{imp}^\pm = P_{marg;post}^\pm - P_{marg;pre}^\pm$ and $P_{imp;\%}^\pm = (P_{marg;post}^\pm - P_{marg;pre})^\pm/P_{marg;pre}^\pm$

- **Slippage**. We define* _slippage_ as the difference between the marginal and the effective price of a trade of a specific size, again either as number or as percentage; specifically, $P_{slip}^\pm(\Delta x) = P_{eff}^\pm(\Delta x) - P_{marg}^\pm$ and $P_{slip\%}^\pm = (P_{eff}^\pm(\Delta x) - P_{marg})^\pm/P_{marg}^\pm$

## Calculations

### Setup

In [2]:
Sim = CarbonSimulatorUI(verbose=False, raiseonerror=False, pair="ETHEURC")
Sim

CarbonSimulatorUI(<0 orders, 0 trades>, pair='ETHEURC')

Add the **ask** positions (AMM sells ETH)

In [3]:
Sim.add_order("ETH", 10, 2000, 2200)
Sim.add_order("ETH", 10, 2005, 2010)
Sim.add_order("ETH", 20, 2000, 2050)
Sim.add_order("ETH", 30, 2020, 2080)
Sim.add_order("ETH", 40, 2030, 2130)
Sim

CarbonSimulatorUI(<5 orders, 0 trades>, pair='ETHEURC')

Add the **bid** positions (AMM buys ETH)

In [4]:
Sim.add_order("EURC", 10000, 1000, 800)
Sim.add_order("EURC", 10000, 995, 990)
Sim.add_order("EURC", 20000, 995, 850)
Sim.add_order("EURC", 30000, 950, 920)
Sim.add_order("EURC", 40000, 970, 870)
Sim

CarbonSimulatorUI(<10 orders, 0 trades>, pair='ETHEURC')

This gives us the following order book

In [5]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,p_start,p_end,p_marg,p_unit,lid
0,0,ETHEURC,ETH,10.0,10.0,ETH,2000.0,2200.0,2000.0,EURC per ETH,0
1,1,ETHEURC,ETH,10.0,10.0,ETH,2005.0,2010.0,2005.0,EURC per ETH,1
2,2,ETHEURC,ETH,20.0,20.0,ETH,2000.0,2050.0,2000.0,EURC per ETH,2
3,3,ETHEURC,ETH,30.0,30.0,ETH,2020.0,2080.0,2020.0,EURC per ETH,3
4,4,ETHEURC,ETH,40.0,40.0,ETH,2030.0,2130.0,2030.0,EURC per ETH,4
5,5,ETHEURC,EURC,10000.0,10000.0,EURC,1000.0,800.0,1000.0,EURC per ETH,5
6,6,ETHEURC,EURC,10000.0,10000.0,EURC,995.0,990.0,995.0,EURC per ETH,6
7,7,ETHEURC,EURC,20000.0,20000.0,EURC,995.0,850.0,995.0,EURC per ETH,7
8,8,ETHEURC,EURC,30000.0,30000.0,EURC,950.0,920.0,950.0,EURC per ETH,8
9,9,ETHEURC,EURC,40000.0,40000.0,EURC,970.0,870.0,970.0,EURC per ETH,9


In [6]:
dxdy_nt = namedtuple("dxdy_nt", "dx,dy")

## Ranges

We now look at the ranges for which we calculate prices. For this, we need a rough estimate for where the current USD price of that asset is. Then we use a range of powers of 10 where the USD value corresponding to the smallest power is between USD 1-10.

In [7]:
current_eth_usd_price = 1500
raw_ranges = ((10**i, i) for i in range(-20,20))
usd_range = ((current_eth_usd_price*x, i) for x,i in raw_ranges)
usd_range = [(x,i) for x,i in usd_range if x>1][0:6]
tkn_range = [10**i for _, i in usd_range]
usd_range, tkn_range

([(1.5, -3), (15.0, -2), (150.0, -1), (1500, 0), (15000, 1), (150000, 2)],
 [0.001, 0.01, 0.1, 1, 10, 100])

## Calculate the amounts

In [8]:
ask = lambda dx: Sim.amm_sells("ETH", dx, execute=False)["trades"].iloc[-1]
bid = lambda dx: Sim.amm_buys("ETH", dx, execute=False)["trades"].iloc[-1]
ask(1)

uid                                     0
id                                      0
subid                                   A
note         AMM sells 1ETH buys 2001EURC
aggr                                 True
exec                                False
limitfail                            None
amt1                                  1.0
tkn1                                  ETH
amt2                          2001.084705
tkn2                                 EURC
pair                              ETHEURC
routeix                            [0, 2]
nroutes                                 2
price                         2001.084705
p_unit                       EURC per ETH
Name: 0, dtype: object

In [9]:
amounts_ask = [dxdy_nt(dx=dx, dy=ask(dx)["amt2"]) for dx in tkn_range]
amounts_ask

[dxdy_nt(dx=0.001, dy=2.000001),
 dxdy_nt(dx=0.01, dy=20.000108),
 dxdy_nt(dx=0.1, dy=200.010842),
 dxdy_nt(dx=1, dy=2001.084705),
 dxdy_nt(dx=10, dy=20056.281233),
 dxdy_nt(dx=100, dy=204852.355)]

In [10]:
amounts_bid = [dxdy_nt(dx=dx, dy=bid(dx)["amt1"]) for dx in tkn_range]
amounts_bid

[dxdy_nt(dx=0.001, dy=0.999989),
 dxdy_nt(dx=0.01, dy=9.998944),
 dxdy_nt(dx=0.1, dy=99.894539),
 dxdy_nt(dx=1, dy=995.460759),
 dxdy_nt(dx=10, dy=9928.869833),
 dxdy_nt(dx=100, dy=93888.325173)]

## Calculate effective prices

Reminder: we calculate the prices for the amounts $10^n, 10^{n+1}, \ldots ETH$ such that $1 USD \leq 10^n ETH < 10 USD$

In [11]:
p_eff_ask = [r.dy/r.dx for r in amounts_ask]
p_eff_ask

[2000.0010000000002,
 2000.0108,
 2000.1084199999998,
 2001.084705,
 2005.6281233000002,
 2048.52355]

In [12]:
p_eff_bid = [r.dy/r.dx for r in amounts_bid]
p_eff_bid

[999.989,
 999.8944,
 998.9453899999999,
 995.460759,
 992.8869833000001,
 938.8832517300001]

## Calculate slippage

Reminder: `slip[n] = price[n] - price[0]` and `slip_pc[n] = (price[n] - price[0])/price[0]`.

Note: take care of the sign adjustment for the bid side! For $x/y$ this adjustment is on the ask side instead.

In [13]:
slip_ask = [f"{p-p_eff_ask[0]:8.4f} USDC/ETH" for p in p_eff_ask]
slip_ask

['  0.0000 USDC/ETH',
 '  0.0098 USDC/ETH',
 '  0.1074 USDC/ETH',
 '  1.0837 USDC/ETH',
 '  5.6271 USDC/ETH',
 ' 48.5225 USDC/ETH']

In [14]:
slip_pc_ask = [f"{(p/p_eff_ask[0]-1)*100:0.4f}%" for p in p_eff_ask]
slip_pc_ask

['0.0000%', '0.0005%', '0.0054%', '0.0542%', '0.2814%', '2.4261%']

In [15]:
slip_bid = [f"{-p+p_eff_bid[0]:8.4f} USDC/ETH" for p in p_eff_bid]
slip_bid

['  0.0000 USDC/ETH',
 '  0.0946 USDC/ETH',
 '  1.0436 USDC/ETH',
 '  4.5282 USDC/ETH',
 '  7.1020 USDC/ETH',
 ' 61.1057 USDC/ETH']

In [16]:
slip_pc_bid = [f"{-(p/p_eff_bid[0]-1)*100:0.4f}%" for p in p_eff_bid]
slip_pc_bid

['-0.0000%', '0.0095%', '0.1044%', '0.4528%', '0.7102%', '6.1106%']